# 1. Установка зависимостей и настройка импорта

In [1]:
# Установка зависимостей
!pip install pandas numpy sentence-transformers scikit-learn requests nltk geopy openrouteservice folium osmnx

In [2]:
# Импорт необходимых библиотек
import os
import time
import zipfile
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import requests
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import openrouteservice
import folium
import osmnx as ox

# 2. Загрузка и обработка данных из Yandex Geo Reviews Dataset

In [3]:
# Скачиваем данные с Kaggle
# Используем идентификатор набора данных: kyakovlev/yandex-geo-reviews-dataset-2023
!kaggle datasets download -d kyakovlev/yandex-geo-reviews-dataset-2023

# Предположим, что файл называется 'yandex-geo-reviews-dataset-2023.zip'
zip_file = 'yandex-geo-reviews-dataset-2023.zip'

# Распаковываем содержимое zip-файла
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('extracted_data')

# Загружаем CSV-файл в DataFrame
# Замените 'geo-reviews-dataset-2023.csv' на фактическое имя CSV-файла внутри архива, если оно другое
csv_file_path = 'extracted_data/geo-reviews-dataset-2023.csv'
df = pd.read_csv(csv_file_path)

# Вывод списка столбцов для диагностики
print("Колонки в загруженном DataFrame:", df.columns)

Dataset URL: https://www.kaggle.com/datasets/kyakovlev/yandex-geo-reviews-dataset-2023
License(s): other
yandex-geo-reviews-dataset-2023.zip: Skipping, found more recently modified local copy (use --force to force download)
Колонки в загруженном DataFrame: Index(['address', 'name_ru', 'rating', 'rubrics', 'text'], dtype='object')


In [4]:
# Маппинг категорий
categories = {
    'развлечения': ['кинотеатр', 'театр', 'концертный зал', 'развлекательный центр', 'аквапарк'],
    'гастрономия': ['ресторан', 'бар, паб', 'кафе', 'кофейня', 'кондитерская', 'быстрое питание', 'пекарня', 'пиццерия', 'столовая'],
    'история': ['музей', 'историческое место', 'монастырь', 'достопримечательность']
}

# Функция для определения категории по рубрикам
def classify_rubrics(rubrics, categories):
    # Список рубрик текущего места
    rubrics_list = [r.strip().lower() for r in rubrics.split(';')]
    # Проходим по всем категориям в поисках первого совпадения ключевого слова с рубрикой
    for category, keywords in categories.items():
        # Если рубрика содержит ключевое слово текущей категории, возвращаем эту категорию
        if any(keyword in rubrics_list for keyword in keywords):
            return category
    # Если ни одно ключевое слово не найдено в рубрике, возвращаем 'неизвестно'
    return 'неизвестно'

# Применяем функцию к каждому отзыву
df['category'] = df['rubrics'].apply(classify_rubrics, categories=categories)

# 3. Подготовка и настройка данных для нейросети

In [5]:
# Загрузка модели
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 4. Обучение модели

In [6]:
# Кодируем вектора
combined_embeddings = model.encode(df['category'].to_list())

# 5. Определение городов и построение маршрутов

In [7]:
 # API-ключ OpenRouteService
API_KEY = "5b3ce3597851110001cf62485690c6ae3c7e4f2a9b1e8ca4f5bfabce"

In [8]:
def get_location_with_address(address):
   # Инициализация геокодера
    geolocator = Nominatim(user_agent="geoapi", timeout=10)

    # Получаем координаты точек
    return geolocator.geocode(address)

In [9]:
def find_cities_on_route(start_address, end_address):
    # Получаем координаты точек
    start_location = get_location_with_address(start_address)
    end_location = get_location_with_address(end_address)

    if not start_location or not end_location:
        raise ValueError("Не удалось получить координаты для одного из адресов.")

    print(f"Стартовые координаты: {start_location.latitude}, {start_location.longitude}")
    print(f"Конечные координаты: {end_location.latitude}, {end_location.longitude}")

    # Создание клиента ORS
    client = openrouteservice.Client(key=API_KEY)

    # Построение маршрута (автомобильный)
    route = client.directions(
        coordinates=[(start_location.longitude, start_location.latitude),
                     (end_location.longitude, end_location.latitude)],
        profile="driving-car",
        format="geojson"
    )

    # Извлечение координат маршрута
    route_coords = [(point[1], point[0]) for point in route["features"][0]["geometry"]["coordinates"]]

    # Инициализация геокодера
    geolocator = Nominatim(user_agent="geoapi", timeout=10)

    # Список для хранения городов на маршруте
    cities_on_route = set()

    # Кэш для координат городов
    city_cache = {}

    # Проход по координатам маршрута, берём каждую 30-ю точку
    for i in range(0, len(route_coords), 30):
        coord = route_coords[i]
        try:
            location = geolocator.reverse(coord, exactly_one=True)
            if location:
                address = location.raw['address']
                city = address.get('city') or address.get('town') or address.get('village')
                if city and city not in cities_on_route:
                    # Проверка расстояния от точки маршрута до центра города
                    if city not in city_cache:
                        city_location = geolocator.geocode(city)
                        if city_location:
                            city_cache[city] = (city_location.latitude, city_location.longitude)
                    if city in city_cache:
                        city_coords = city_cache[city]
                        distance = geodesic(coord, city_coords).km
                        if distance <= 30:  # Радиус поиска
                            cities_on_route.add((city, city_coords))
        except Exception as e:
            print(f"Ошибка геокодирования: {e}")
        time.sleep(0.1)  # Уменьшите задержку между запросами

    # Возвращаем список городов, а также стартовые и конечные координаты
    return list(cities_on_route), (start_location.latitude, start_location.longitude), (end_location.latitude, end_location.longitude)

In [10]:
def create_route_map(start_location, end_location, cities_on_route, start_address, end_address):
    # Создание клиента ORS
    client = openrouteservice.Client(key=API_KEY)

    # Подготовка списка координат для маршрута
    waypoints = [(start_location[1], start_location[0])]  # Переворачиваем координаты для ORS
    waypoints.extend([(city[1][1], city[1][0]) for city in cities_on_route])  # Добавляем города
    waypoints.append((end_location[1], end_location[0]))  # Конечная точка

    # Построение маршрута (автомобильный)
    route = client.directions(
        coordinates=waypoints,
        profile="driving-car",
        format="geojson"
    )

    # Извлечение координат маршрута
    route_coords = [(point[1], point[0]) for point in route["features"][0]["geometry"]["coordinates"]]

    # Создание карты с центром в стартовой точке
    m = folium.Map(location=start_location, zoom_start=6)

    # Добавление маркеров
    folium.Marker(start_location,
        popup=f"<b>Старт:</b> {start_address}",
        icon=folium.Icon(color="green", icon="play")).add_to(m)

    for city_name, city_coords in cities_on_route:
        folium.Marker(city_coords,
            popup=f"<b>Город:</b> {city_name}",
            icon=folium.Icon(color="blue", icon="info-sign")).add_to(m)

    folium.Marker(end_location,
        popup=f"<b>Финиш:</b> {end_address}",
        icon=folium.Icon(color="red", icon="stop")).add_to(m)

    # Добавление маршрута на карту
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.7, tooltip="Маршрут").add_to(m)

    # Сохранение карты в файл
    m.save("route_map.html")

    return m

# 6. Генерация рекомендаций на основе модели

In [11]:
def get_top_places(query_embedding, cities_on_route, n_per_city=10):
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding.reshape(1, -1)

    similarity_scores = cosine_similarity(query_embedding, combined_embeddings).flatten()
    df_sims = df.copy()
    df_sims['similarity'] = similarity_scores

    # Расчёт средних показателей рейтинга и "схожести" для каждого места
    df_sims = df_sims.groupby(['address', 'name_ru', 'category'], as_index=False).agg({
        'rating': 'mean',      # Среднее рейтинга
        'similarity': 'mean'   # Среднее "схожести"
    })

    # Массив случайных коэффициентов
    random_coefs = np.random.uniform(0.9, 1.1, size=len(df_sims))

    # Расчёт итогового балла для каждого места
    df_sims['score'] = df_sims['rating'] * df_sims['similarity'] * random_coefs

    # Сортировка по убыванию итогового балла
    top_places = df_sims.sort_values(by='score', ascending=False)

    # Отладочный вывод количества всех топ мест
    print(f"Отладка: количество всех топ мест отобранных по косинусному сходству: {len(df_sims)}\n")

    recommendations_by_city = {}
    for city_info in cities_on_route:
        city_name = city_info[0].strip().lower()

        # Отладочный вывод
        print(f"Ищем места для города: {city_name}")

        # Фильтрация по городу
        city_places = top_places[top_places['address'].str.contains(city_name, case=False, na=False)]

        # Отладочный вывод
        if city_places.empty:
            print(f"Нет совпадений для города: {city_name}")
        else:
            print(f"Найдено {len(city_places)} совпадений для города: {city_name}")

        if not city_places.empty:
            # Отбор топ мест
            recommendations_by_city[city_name] = city_places.head(n_per_city)[['name_ru', 'address', 'rating', 'category']]

    return recommendations_by_city

In [14]:
def display_top_places(recommendations_by_city):
    print("🏆 Топовые места для вашего запроса: 🏆\n")

    for city, top_places in recommendations_by_city.items():
        print(f"🌆 Город: {city.title()}\n")

        for index, place in top_places.iterrows():
            name = place['name_ru']
            rating = place['rating']
            category = place['category']
            address = place['address']

            print(f"📍 Место: {name}")
            print(f"⭐ Рейтинг: {rating}")
            print(f"🔖 Категория: {category}")
            print(f"📫 Адрес: {address}\n")
            print("-" * 40)

        print("\n" + "=" * 50 + "\n")

# 7. Предоставление тестовых данных и вывод результатов

In [12]:
# Ввод адресов для построения маршрута
start_address = "Москва, Россия"
end_address = "Казань, Россия"

# Поиск городов по маршруту
cities, start_coords, end_coords = find_cities_on_route(start_address, end_address)

# Смотрим результат выполнения поиска по городам
print(cities)

Стартовые координаты: 55.625578, 37.6063916
Конечные координаты: 55.7823547, 49.1242266
[('Москва', (55.625578, 37.6063916)), ('Владимир', (56.1288899, 40.4075203)), ('городской округ Казань', (55.7823547, 49.1242266)), ('Казань', (55.7823547, 49.1242266)), ('Верхний Услон', (55.7685496, 48.9829213))]


In [15]:
# Запрос пользователя
user_query = "Какие музеи посетить и где вкусно поесть?"

# Кодирование пользовательского запроса
user_query_embedding = model.encode(user_query).reshape(1, -1)

# Получение топ мест для пользовательского запроса
top_places_user = get_top_places(user_query_embedding, cities)

# Вывод результатов
print("\nТоп популярных мест для вашего запроса:")
display_top_places(top_places_user)


# Строим и показываем карту
map = create_route_map(start_coords, end_coords, cities, start_address, end_address)
map

Отладка: количество всех топ мест отобранных по косинусному сходству: 275816

Ищем места для города: москва
Найдено 49644 совпадений для города: москва
Ищем места для города: владимир
Найдено 3209 совпадений для города: владимир
Ищем места для города: городской округ казань
Нет совпадений для города: городской округ казань
Ищем места для города: казань
Найдено 3884 совпадений для города: казань
Ищем места для города: верхний услон
Найдено 2 совпадений для города: верхний услон

Топ популярных мест для вашего запроса:
🏆 Топовые места для вашего запроса: 🏆

🌆 Город: Москва

📍 Место: Drive City
⭐ Рейтинг: 5.0
🔖 Категория: развлечения
📫 Адрес: Москва, Поречная улица, 10

----------------------------------------
📍 Место: КлаусКидс
⭐ Рейтинг: 5.0
🔖 Категория: развлечения
📫 Адрес: Москва, улица Авиаконструктора Микояна, 10

----------------------------------------
📍 Место: Московский драматический театр Человек
⭐ Рейтинг: 5.0
🔖 Категория: развлечения
📫 Адрес: Москва, Скатертный переулок, 23А
